In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
import datetime
from utils import DailySeries2Features, train_val_test_purge_combinatorial_kfold, train_val_test

In [2]:
root = os.getcwd()

In [3]:
print(root)

C:\Users\micha\OneDrive\Documents\CSCI E-599a Data Science Capstone\capstone


In [4]:
data_env = root+"/data_env/"

In [5]:
os.listdir(data_env)

['EEMV.parquet',
 'HYG.parquet',
 'MTUM.parquet',
 'QUAL.parquet',
 'SIZE.parquet',
 'USMV.parquet',
 'VLUE.parquet']

In [6]:
# read
df = pd.read_parquet(data_env+'EEMV.parquet')

# # write
# df.to_parquet('my_newfile.parquet')

df

,open,high,low,close,volume
index,,,,,
2011-10-20 04:00:00+00:00,39.27172,39.27172,38.71069,38.97518,1372
2011-10-21 04:00:00+00:00,39.48811,39.63238,39.48811,39.63238,4242
2011-10-25 04:00:00+00:00,40.65825,40.79450,40.65825,40.72237,3493
2011-10-26 04:00:00+00:00,40.83457,40.93074,40.83457,40.88266,374
2011-10-27 04:00:00+00:00,42.07684,42.73404,42.07684,42.51764,1622
...,...,...,...,...,...
2020-11-17 05:00:00+00:00,59.16000,59.61500,59.14500,59.46000,265100
2020-11-18 05:00:00+00:00,59.50000,59.67000,59.36000,59.36000,735100
2020-11-19 05:00:00+00:00,59.15000,59.50840,59.07010,59.41000,170500


In [7]:
assets_dict = {file: pd.read_parquet(data_env + "/" + file) for file in os.listdir(data_env)}
counter=0
for key, value in assets_dict.items():
    if counter==0:
        main_index=value.index
    else:
        main_index=main_index.join(value.index,how="inner")

for key, value in assets_dict.items():
    tmp_df=value.reindex(main_index)
    tmp_df=tmp_df.fillna(method='ffill')
    assets_dict[key]=tmp_df['close']

{'EEMV.parquet': index
 2013-04-18 04:00:00+00:00    49.95146
 2013-04-24 04:00:00+00:00    50.74618
 2013-04-25 04:00:00+00:00    51.04420
 2013-04-26 04:00:00+00:00    50.87036
 2013-04-30 04:00:00+00:00    51.79753
                                ...   
 2020-11-17 05:00:00+00:00    59.46000
 2020-11-18 05:00:00+00:00    59.36000
 2020-11-19 05:00:00+00:00    59.41000
 2020-11-20 05:00:00+00:00    59.62000
 2020-11-23 05:00:00+00:00    59.73000
 Name: close, Length: 1857, dtype: float64,
 'HYG.parquet': index
 2013-04-18 04:00:00+00:00    62.96130
 2013-04-24 04:00:00+00:00    63.43434
 2013-04-25 04:00:00+00:00    63.60756
 2013-04-26 04:00:00+00:00    63.54094
 2013-04-30 04:00:00+00:00    63.86074
                                ...   
 2020-11-17 05:00:00+00:00    85.95000
 2020-11-18 05:00:00+00:00    85.79000
 2020-11-19 05:00:00+00:00    86.08000
 2020-11-20 05:00:00+00:00    85.92000
 2020-11-23 05:00:00+00:00    86.05000
 Name: close, Length: 1857, dtype: float64,
 'MTUM.pa

In [8]:
portfolio_df = pd.DataFrame()

for key, value in assets_dict.items():
    key = key.split(".")[0]
    tmp_df = pd.DataFrame(data=value)
    tmp_df.columns=[key]
    portfolio_df = pd.concat([portfolio_df, tmp_df], axis=1)

portfolio_df.index = pd.to_datetime(portfolio_df.index, errors='coerce').tz_localize(None)
# portfolio_df=portfolio_df[portfolio_df.index>'2014-07-18']
portfolio_df

,EEMV,HYG,MTUM,QUAL,SIZE,USMV,VLUE
index,,,,,,,
2013-04-18 04:00:00,49.95146,62.96130,46.29308,NaN,43.54477,28.10539,41.37362
2013-04-24 04:00:00,50.74618,63.43434,46.87492,NaN,43.95286,28.52423,41.96898
2013-04-25 04:00:00,51.04420,63.60756,47.10221,NaN,44.47384,28.64390,42.77398
2013-04-26 04:00:00,50.87036,63.54094,47.06584,NaN,44.45647,28.60972,42.58950
2013-04-30 04:00:00,51.79753,63.86074,47.12948,NaN,44.82116,28.78067,41.90190
...,...,...,...,...,...,...,...
2020-11-17 05:00:00,59.46000,85.95000,151.45000,112.47,105.71000,67.20000,83.14000
2020-11-18 05:00:00,59.36000,85.79000,150.73000,111.14,104.57000,66.35000,82.63000
2020-11-19 05:00:00,59.41000,86.08000,151.95000,111.29,105.26000,66.47000,82.89000


In [14]:
y = pd.Series(None)
forward_return_date = pd.Timedelta(days=1)
return_dates = pd.Series(portfolio_df.index) + forward_return_date

return_dates

# eval_times_df = pd.Series(return_dates['time'].values, index = return_dates.index)
# eval_times_df=pd.to_datetime(eval_times_df)
# eval_times_df.index = pd.to_datetime(eval_times_df.index).tz_localize(None)
# eval_times_df.tz_localize(None)
# eval_times_df
# weights = 0

0      2013-04-19 04:00:00
1      2013-04-25 04:00:00
2      2013-04-26 04:00:00
3      2013-04-27 04:00:00
4      2013-05-01 04:00:00
               ...        
1852   2020-11-18 05:00:00
1853   2020-11-19 05:00:00
1854   2020-11-20 05:00:00
1855   2020-11-21 05:00:00
1856   2020-11-24 05:00:00
Name: index, Length: 1857, dtype: datetime64[ns]

In [15]:
last_index, embargo_td = train_val_test(portfolio_df, y, return_dates)

TRAIN                          EEMV       HYG       MTUM      QUAL      SIZE  \
index                                                                    
2013-04-18 04:00:00  49.95146  62.96130   46.29308       NaN  43.54477   
2013-04-24 04:00:00  50.74618  63.43434   46.87492       NaN  43.95286   
2013-04-25 04:00:00  51.04420  63.60756   47.10221       NaN  44.47384   
2013-04-26 04:00:00  50.87036  63.54094   47.06584       NaN  44.45647   
2013-04-30 04:00:00  51.79753  63.86074   47.12948       NaN  44.82116   
...                       ...       ...        ...       ...       ...   
2019-05-29 04:00:00  54.39727  79.08764  109.91950  84.29336  83.58955   
2019-05-30 04:00:00  54.71522  79.25446  110.76220  84.50752  83.68686   
2019-05-31 04:00:00  55.01389  78.67989  109.92930  83.58272  82.74294   
2019-06-03 04:00:00  55.52453  78.80650  109.18460  83.31988  83.08353   
2019-06-04 04:00:00  55.67869  79.59782  111.31090  85.28631  85.09785   

                         USMV  

In [16]:
assets_dict = {file: pd.read_parquet(data_env + "/" + file) for file in os.listdir(data_env)}

In [19]:
for file in os.listdir(data_env):
    tmp_df = pd.read_parquet(data_env + "/" + file)

    tmp_df_train = tmp_df.iloc[:last_index-embargo_td]
    tmp_df_train.to_parquet("data_env_train/" + file)
    tmp_df_test = tmp_df.iloc[last_index:] 
    tmp_df_test.to_parquet("data_env_test/" + file)